Projet Final

In [1]:
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup

Aquisition des données

In [3]:

url_template = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_{}_{}.xlsx"

begin_datetime = datetime.datetime(year=2020, month=11, day=8, hour=14)
end_datetime = datetime.datetime(year=2021, month=3, day=5, hour=8)
time_delta = datetime.timedelta(hours=1)

files_count = 0
while begin_datetime <= end_datetime:
    try:
        url = url_template.format(begin_datetime.date().strftime("%Y%m%d"), begin_datetime.time().strftime("%H%M%S"))
        r = requests.get(url)
        if r.status_code == requests.codes.ok:
            files_count +=1
            with open("./datasets/"+url.split('/')[-1], 'wb') as f:
                f.write(r.content)
    except:
        pass
    begin_datetime += time_delta

print("{} files downloaded".format(files_count))


702 files downloaded


In [100]:
df = pd.read_excel("datasets/vendeeglobe_20201108_140000.xlsx", header=3, engine="openpyxl", usecols="B:")
#import openpyxl
#wb = openpyxl.load_workbook(filename= "datasets/vendeeglobe_20201108_210000.xlsx")
df.head()

""


In [48]:
r = requests.get("https://www.vendeeglobe.org/fr/glossaire")
soup = BeautifulSoup(r.content, "html.parser")
soup.prettify()
specs = []
for s in soup.find_all("div", {"class": "boats-list__popup-infos"}):
    d = {}
    d["Bateau"] = s.find("h3", {"class": "boats-list__popup-title"}).text
    for l in s.find_all("li"):
        text = l.text.split(':')
        d[text[0].strip()] = text[1].strip()
    specs.append(d)
df_spec = pd.DataFrame(specs)
#df_spec["Date de lancement"] = pd.to_datetime(df_spec["Date de lancement"] , format="%d %B %Y")
df_spec["Longueur"] = df_spec["Longueur"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()).astype(float)
df_spec["Largeur"] = df_spec["Largeur"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()).astype(float)
df_spec["Tirant d'eau"] = df_spec["Tirant d'eau"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()).astype(float)
df_spec["Déplacement (poids)"] = pd.to_numeric(df_spec["Déplacement (poids)"].apply(lambda x: x.split('t')[0].replace(',', '.').strip()), errors="coerce")
df_spec["Hauteur mât"] = pd.to_numeric(df_spec["Hauteur mât"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()), errors="coerce")
df_spec["Surface de voiles au près"] = pd.to_numeric(df_spec["Surface de voiles au près"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()), errors="coerce")
df_spec["Surface de voiles au portant"] = pd.to_numeric(df_spec["Surface de voiles au portant"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()), errors="coerce")
df_spec = df_spec.rename(columns={"Longueur": "Longeur (m)", "Largeur": "Largeur (m)", "Tirant d'eau": "Tirant d'eau (m)", "Déplacement (poids)": "Déplacement (poids en tonne)",\
     "Hauteur mât":"Hauteur mât (m)", "Surface de voiles au près": "Surface de voiles au près (m2)", "Surface de voiles au portant": "Surface de voiles au portant (m2)"})

df_spec

,Bateau,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longeur (m),Largeur (m),Tirant d'eau (m),Déplacement (poids en tonne),Nombre de dérives,Hauteur mât (m),Voile quille,Surface de voiles au près (m2),Surface de voiles au portant (m2)
0,NEWREST - ART & FENÊTRES,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,18.28,5.85,4.5,7.0,foils,29.0,monotype,320,570
1,PURE - Best Western®,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,18.28,5.80,4.5,9.0,2,28.0,acier forgé,280,560
2,TSE - 4MYPLANET,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,18.28,5.54,4.5,9.0,2,29.0,acier,260,580
3,Maître CoQ IV,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,18.28,5.80,4.5,8.0,foils,29.0,acier mécano soudé,310,550
4,CHARAL,08,NaN,VPLP,CDK Technologies,18 Août 2018,18.28,5.85,4.5,8.0,foils,29.0,acier,320,600
5,LA MIE CÂLINE - ARTISANS ARTIPÔLE,FRA 14,"Ecover3, Président, Gamesa, Kilcullen Voyager-...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,18.28,5.65,4.5,7.9,foils,29.0,basculante avec vérin,300,610
6,BUREAU VALLEE 2,18,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,18.28,5.80,4.5,7.6,foils,28.0,acier,300,600
7,ONE PLANET ONE OCEAN,ESP 33,Kingfisher - Educacion sin Fronteras - Forum M...,Owen Clarke Design,Martens Yachts,02 Février 2000,18.28,5.30,4.5,8.9,2,26.0,acier,240,470
8,GROUPE SÉTIN,FRA 71,"Paprec-Virbac2, Estrella Damm, We are Water, L...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,18.28,5.80,4.5,9.0,2 asymétriques,28.5,basculante sur vérin hydraulique,270,560
9,BANQUE POPULAIRE X,FRA30,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,18.28,5.70,4.5,7.7,2,29.0,acier forgé,340,570
